<a href="https://colab.research.google.com/github/kahyunp/viva/blob/main/SW%EB%AC%B8%EC%A0%9C%EC%98%81%EC%97%AD%EC%9E%90%EB%A5%B4%EA%B8%B0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
! apt install tesseract-ocr
! apt install libtesseract-dev
! pip install Pillow
! pip install pytesseract
! sudo apt-get install tesseract-ocr-script-hang tesseract-ocr-script-hang-vert

In [ ]:
def cropTop(image):
    img=cv2.imread(image)
    #cv2_imshow(src)
    dst = img.copy()
    h,w,_=img.shape
    cv2_imshow(img)
    #이미지 흑백화 
    imgray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY) 
    img2=imgray.copy()

    #이미지 이진화 (스캔본 처럼)
    blur = cv2.GaussianBlur(imgray, ksize=(3,3), sigmaX=0)
    thresh = cv2.threshold(blur, 0, 255, cv2.THRESH_BINARY+cv2.THRESH_OTSU)[1]

    # Morph operations
    edge = cv2.Canny(imgray, 100, 200)
    kernel = cv2.getStructuringElement(cv2.MORPH_RECT,(100,100))
    closed = cv2.morphologyEx(edge, cv2.MORPH_CLOSE, kernel)

	
    contours, hierarchy = cv2.findContours(closed.copy(),cv2.RETR_TREE, cv2.CHAIN_APPROX_NONE)

    #cv2.drawContours(img,contours,-1,(255,0,0),10)
    #cv2_imshow(img)
    #경계를 직사각형으로 찾기
    rects = [cv2.boundingRect(each) for each in contours]
  
    page=sorted(rects, key= lambda x: x[1])[-1]
    body=max(rects, key= lambda x: x[2]*x[3])
    length=w-page[1]


    
    #cv2.rectangle(img,(body[0],body[1]),(body[0]+body[2],body[1]+body[3]),(255,255,0),13)
    #cv2_imshow(img)    
    margin=15
    
    bottom=imgray[page[1]-margin:,:]
    blank=bottom.shape[0]
    # print(blank)
    # print(body)
    content=imgray[body[1]+blank-margin:body[1]+body[3],body[0]-margin:body[0]+body[2]+margin]
    half=int(content.shape[1]/2)
    left=content[:,:half-margin]
    right=content[:,half+margin:]
    #cv2_imshow(bottom)
    contour(left)
    contour(right)
    # print(len(rects))

    #print(miny)
    #cv2_imshow(body)
        
    return header,body
      
     
       



In [ ]:
#@title
import cv2
import numpy as np
from google.colab.patches import cv2_imshow
import glob
import os

from PIL import ImageEnhance, ImageFilter, Image
if __name__ == "__main__":
    
    os.chdir('/content/drive/MyDrive/GRADING_Study/kh/swraw/')
    images=glob.glob('./*.png')
    images.sort()
    print(images)
    qnum=0
    for i in range(len(images)):
        filename=images[i]
        print(filename)
        cropTop(filename)
       
        #cropCenter(filename)
        #readPage(filename[-6:],header)

In [ ]:

#반페이지를 입력받고 크롭하기 
def contour(page_rl):
    import pytesseract

    print("contour")

    last=page_rl.shape[0]

    #이미지 흑백화 
    imgray = page_rl.copy()
    #이미지 이진화 (스캔본 처럼)
    blur = cv2.GaussianBlur(imgray, (3,3), 0)
    thresh = cv2.threshold(blur, 0, 255, cv2.THRESH_BINARY+cv2.THRESH_OTSU)[1]


    # Morph operations
    edge = cv2.Canny(imgray, 100, 200)
    kernel = cv2.getStructuringElement(cv2.MORPH_RECT,(1000,100))
    closed = cv2.morphologyEx(edge, cv2.MORPH_CLOSE, kernel)

	
    contours, hierarchy = cv2.findContours(closed.copy(),cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    
    
    #한페이지 내에서 문제 순서대로 불러오기
    contours=reversed(contours)

    #cv2.drawContours(page_rl,contours,-1,(0,255,0),10)
    #cv2_imshow(imgray)
	#한페이지 내의 모든 폐곡선 범위에 대해 실행 
    top=[]
    for c in contours:

		#폐곡선 바운더리 
        x,y,w,h = cv2.boundingRect(c)
        top.append(y)
    top.append(last)
    

    # print(top)
    total=len(top)-1
    # global qnum
    for i in range(total):
        print("Q")
        if (i==0):
            img_trim=page_rl[0:top[i+1]-5,:]
        else:
            img_trim=page_rl[top[i]-10:top[i+1]-5,:]
        #cv2.imwrite('/content/MyDrive/GRADIING_Study/kh/trim/'+str(qnum)+'.png',img_trim)
        d=pytesseract.image_to_string(img_trim)
        d=d.replace(' ','')
        book_serial='20050-'
        if (book_serial in d):
            start=d.find(book_serial)
            serial=d[start:start+10]
            print(serial)
            cv2_imshow(img_trim)

In [ ]:
def cropCenter(img):
    src=cv2.imread(img)
    dst = src.copy()
    gray = cv2.cvtColor(src, cv2.COLOR_BGR2GRAY)
    canny = cv2.Canny(gray, 5000, 1500, apertureSize = 5, L2gradient = True)
    lines = cv2.HoughLines(canny, 0.8, np.pi / 180, 150, srn = 100, stn = 200, min_theta = 0, max_theta = np.pi/8)
    row=0
    col=0
    for i in lines:
        rho, theta = i[0][0], i[0][1]
        a, b = np.cos(theta), np.sin(theta)
        x0, y0 = a*rho, b*rho
        print("x0",x0,"y0",y0)
        scale = src.shape[0] + src.shape[1]

        x1 = int(x0 + scale * -b)
        y1 = int(y0 + scale * a)
        x2 = int(x0 - scale * -b)
        y2 = int(y0 - scale * a)
        row=x2
        col=y2
        cv2.line(dst, (x1, y1), (x2, y2), (0, 0, 255), 2)
        cv2.circle(dst, (x0, y0), 3, (255, 0, 0), 1, cv2.FILLED)

    #left = src[:, :row-15]
    #right= src[:,row+15:]
   
    cv2_imshow(dst)
    #contour(left)
    #contour(right)

In [ ]:

#반페이지를 입력받고 크롭하기 
def contour(page_rl):
    print("contour")

    #이미지 흑백화 
    imgray = cv2.cvtColor(page_rl, cv2.COLOR_BGR2GRAY) 
    img2=imgray.copy()
    #이미지 이진화 (스캔본 처럼)
    blur = cv2.GaussianBlur(imgray, (3,3), 0)
    thresh = cv2.threshold(blur, 70, 255, cv2.THRESH_BINARY+cv2.THRESH_OTSU)[1]


    # Morph operations
    edge = cv2.Canny(imgray, 100, 200)
    kernel = cv2.getStructuringElement(cv2.MORPH_RECT,(1000,200))
    closed = cv2.morphologyEx(edge, cv2.MORPH_CLOSE, kernel)

	
    contours, hierarchy = cv2.findContours(closed.copy(),cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    

    contours_xy = np.array(contours , dtype=object)
    contours_xy.shape
    #한페이지 내에서 문제 순서대로 불러오기
    contours=reversed(contours)


	#한페이지 내의 모든 폐곡선 범위에 대해 실행 
    top=[]
    for c in contours:

		#폐곡선 바운더리 
        x,y,w,h = cv2.boundingRect(c)
        top.append(y)
        #cv2.rectangle(page_rl, (x,y), (w,h), (0,0,255), 3) 
        #img_trim = page_rl[y:y+h+20 , :]
        #cv2_imshow(img_trim)
            #cv2.imwrite('/content/drive/MyDrive/GRADING_Study/kh/trim/'+str (qnum)+'.png',img_trim)
            #cv2_imshow(page_rl)
            #include(img_trim)
    print(top)
    total=len(top)-1
    global qnum
    for i in range(total):
        qnum+=1
        if (i==0):
            img_trim=page_rl[top[i]:top[i+1]-5,:]
        else:
            img_trim=page_rl[top[i]-10:top[i+1]-5,:]
        #cv2.imwrite('/content/MyDrive/GRADIING_Study/kh/trim/'+str(qnum)+'.png',img_trim)
        print(qnum)
        #cv2_imshow(img_trim)

In [ ]:
def readPage(name,img):
    path='/content/drive/MyDrive/GRADING_Study/kh/header/'
    cv2_imshow(img)
    print(path+name)
    cv2.imwrite(path+name, img)

In [ ]:
def cropCenter(img):
    src=img
    dst=src.copy
    w=int (img.shape[1]/2)
    left=src[:,:w-5]
    right=src[:,w+5:]
    #cv2_imshow(left)
    #cv2_imshow(right)
    contour(left)
    contour(right)


In [ ]:

def include(cropped):
  
    d = pytesseract.image_to_string(cropped, lang='eng+Hangul')
    d=d.replace(' ','')
    print(d)